In [1]:
import pandas as pd
import glob
from transformers import GPT2Tokenizer
import tiktoken
import numpy as np
import torch
from transformers import pipeline

/home/vamsi/Desktop/projects/Finetuning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df=pd.read_csv('master.csv')

In [3]:
# df.head()

In [4]:
# df2=pd.concat([df['Question']+df['Answer']],ignore_index=True)

In [5]:
# df2=" {Question} : "+df['Question']+" {Answer} : "+df['Answer']

In [6]:
# df2.iloc[0]

In [7]:
# df2.shape

In [2]:
enc=tiktoken.get_encoding('gpt2')

In [3]:
# txt=''
# for i in range(df2.shape[0]):
#     txt=txt+df2.iloc[i]

In [5]:
# tokens=[]
# tokens.extend(enc.encode(txt))
# tokens_np=np.array(tokens)
# tokens_np_uint16=tokens_np.astype(np.uint16)
# np.save('numpy_arrays/data',tokens_np_uint16)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
buf=np.load('numpy_arrays/data.npy')
buf = buf.astype(np.int32)
buf=torch.tensor(buf,dtype=torch.long)

In [9]:
from transformers import GPT2LMHeadModel
from torch.optim import AdamW

/home/vamsi/Desktop/projects/Finetuning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
int(buf.shape[0]*0.9)-int(buf.shape[0]*0.8)

1067329

In [4]:
train_buf=buf[:int(buf.shape[0]*0.8)]
val_buf=buf[int(buf.shape[0]*0.8):int(buf.shape[0]*0.9)]
test_buf=buf[int(buf.shape[0]*0.9):]

In [12]:
model_hf=GPT2LMHeadModel.from_pretrained('gpt2')

In [13]:
def getxys(buf,iter,B,T):
    
    if(B*T*(iter+1)+1>buf.shape[0]):
        pool=buf[B*T*iter:]+buf[:(B*T*(iter+1))-buf.shape[0]+1]    
    else:
        pool=buf[B*T*iter:B*T*(iter+1)+1]
    x=(pool[:-1]).view(B,T)
    y=(pool[1:]).view(B,T)
    return x,y

In [7]:
B=1
T=512
max_iter=20846 #buf.shape[0]//(B*T)

In [15]:
# x,y=getxys(buf,0,B,T)

In [16]:
# x=x.to(device)
# y=y.to(device)

In [17]:
# x.shape

In [18]:
# model_hf=model_hf.to(device)

In [19]:
type(model_hf)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [20]:
model_hf=model_hf.to(device)

In [21]:
model_hf.state_dict()

OrderedDict([('transformer.wte.weight',
              tensor([[-0.1101, -0.0393,  0.0331,  ..., -0.1364,  0.0151,  0.0453],
                      [ 0.0403, -0.0486,  0.0462,  ...,  0.0861,  0.0025,  0.0432],
                      [-0.1275,  0.0479,  0.1841,  ...,  0.0899, -0.1297, -0.0879],
                      ...,
                      [-0.0445, -0.0548,  0.0123,  ...,  0.1044,  0.0978, -0.0695],
                      [ 0.1860,  0.0167,  0.0461,  ..., -0.0963,  0.0785, -0.0225],
                      [ 0.0514, -0.0277,  0.0499,  ...,  0.0070,  0.1552,  0.1207]],
                     device='cuda:0')),
             ('transformer.wpe.weight',
              tensor([[-1.8821e-02, -1.9742e-01,  4.0267e-03,  ..., -4.3044e-02,
                        2.8267e-02,  5.4490e-02],
                      [ 2.3959e-02, -5.3792e-02, -9.4879e-02,  ...,  3.4170e-02,
                        1.0172e-02, -1.5573e-04],
                      [ 4.2161e-03, -8.4764e-02,  5.4515e-02,  ...,  1.9745e-02,
     

In [22]:
optimizer=AdamW(params= model_hf.parameters(),lr=6e-5)

In [23]:
val_iter=0
for iter in range(max_iter):
    model_hf.train()
    optimizer.zero_grad()
    x,y=getxys(train_buf,iter,B,T)
    x=x.to(device)
    y=y.to(device)
    op=model_hf(input_ids=x,labels=y)
    # op.loss.retain_grad()
    op.loss.backward()
    optimizer.step()
    print(f'train_loss at iter {iter}:{op.loss.detach():.4f}')
    del x,y
    torch.cuda.empty_cache()
    if(iter%100==0):
        x_val,y_val=getxys(val_buf,val_iter,B,T)
        x_val,y_val=x_val.to(device),y_val.to(device)
        val_iter+=1
        model_hf.eval()
        op=model_hf(input_ids=x_val,labels=y_val)
        print(f'val_loss : {op.loss.detach():.4f}')

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


train_loss at iter 0:9.5751
val_loss : 8.8989
train_loss at iter 1:7.9608
train_loss at iter 2:7.2049
train_loss at iter 3:6.6801
train_loss at iter 4:6.7960
train_loss at iter 5:6.5739
train_loss at iter 6:6.2667
train_loss at iter 7:6.8739
train_loss at iter 8:6.7757
train_loss at iter 9:6.7904
train_loss at iter 10:6.5886
train_loss at iter 11:6.2401
train_loss at iter 12:6.3268
train_loss at iter 13:6.1809
train_loss at iter 14:6.3629
train_loss at iter 15:5.9953
train_loss at iter 16:6.2674
train_loss at iter 17:6.0693
train_loss at iter 18:6.1452
train_loss at iter 19:5.9488
train_loss at iter 20:5.8246
train_loss at iter 21:5.6617
train_loss at iter 22:5.9797
train_loss at iter 23:5.9510
train_loss at iter 24:5.3167
train_loss at iter 25:5.8081
train_loss at iter 26:5.5955
train_loss at iter 27:5.4983
train_loss at iter 28:5.4283
train_loss at iter 29:5.7829
train_loss at iter 30:4.6925
train_loss at iter 31:5.7757
train_loss at iter 32:5.5814
train_loss at iter 33:5.6524
train_

RuntimeError: The size of tensor a (11) must match the size of tensor b (502) at non-singleton dimension 0

In [24]:
model_hf.state_dict()

OrderedDict([('transformer.wte.weight',
              tensor([[-0.1126, -0.0625,  0.1289,  ..., -0.1434,  0.0204, -0.0416],
                      [ 0.0330, -0.0667,  0.0750,  ...,  0.0607,  0.0055,  0.0323],
                      [-0.1131, -0.0275,  0.2418,  ...,  0.0677, -0.1407, -0.1738],
                      ...,
                      [-0.0624, -0.0955,  0.0952,  ...,  0.0436,  0.0849, -0.1267],
                      [ 0.1555, -0.0363,  0.1373,  ..., -0.1186,  0.0366, -0.1196],
                      [ 0.0447, -0.1094,  0.1807,  ..., -0.0358,  0.1540, -0.0417]],
                     device='cuda:0')),
             ('transformer.wpe.weight',
              tensor([[-0.0268, -0.1914, -0.0008,  ..., -0.0425,  0.0374,  0.0513],
                      [ 0.0353, -0.0356, -0.0948,  ...,  0.0381,  0.0038, -0.0055],
                      [ 0.0071, -0.0813,  0.0437,  ...,  0.0154,  0.0102, -0.0207],
                      ...,
                      [-0.0018,  0.0016, -0.0545,  ...,  0.0135, -0.0

In [25]:
model_hf._save_to_state_dict('model/model_dict.pth')

TypeError: Module._save_to_state_dict() missing 2 required positional arguments: 'prefix' and 'keep_vars'

In [5]:
model_hf_imp=torch.load('model/entire_model.pth',weights_only=False)

In [ ]:
# torch.save(model_hf,"model/entire_model.pth")

In [13]:
a=enc.encode('{Question} : Is keratoderma with woolly hair inherited ?')

In [14]:
a=torch.tensor(a)

In [16]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [35]:
# opt=model_hf.generate(a,max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)
# generated_text=enc.decode(opt[0])
# generated_text

In [ ]:

generator = pipeline("text-generation", model=model_hf_imp, tokenizer=tokenizer)

# Generate text
generated_text = generator("{Question} : Is keratoderma with woolly hair inherited ?", max_length=100, num_return_sequences=1)


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [19]:
print(generated_text[0]['generated_text'])

{Question} : Is keratoderma with woolly hair inherited ?Answer : areatmathe skin. signs symptoms vary  age aic? The isonia that with of hair in individuals ashen, or, are not is characteristic people this- age. most of symptoms be byement  may a,,,, be,,., theonia other, is of,. oror It be or:/ may as, a, occur a, a, a an,


In [13]:
2*512

1024

In [14]:
x=(torch.tensor(test_buf[:1024])).view(2,T)

/tmp/ipykernel_659200/608091567.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=(torch.tensor(test_buf[:1024])).view(2,T)


In [16]:
y=(torch.tensor(test_buf[1:1025])).view(2,T)

/tmp/ipykernel_659200/2453114930.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=(torch.tensor(test_buf[1:1025])).view(2,T)


In [17]:
x,y=x.to(device),y.to(device)

In [18]:
model_hf_imp.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [20]:
OPT=model_hf_imp(input_ids=x,labels=y)

In [21]:
OPT.loss

tensor(4.9418, device='cuda:0', grad_fn=<NllLossBackward0>)